In [ ]:
"""
important param:
app_id
app_secret
headers = {‘Authorization’: f’Bearer {get_token()}', ## 获取tenant_access_token, 需要替换为实际的token
"""

In [2]:
import requests
from requests_toolbelt import MultipartEncoder
import json
import logging
import logger

In [ ]:
def get_token():
    # 获取tenant_access_token，供上传图片接口使用
    url = "https://open.feishu.cn/open-apis/auth/v3/tenant_access_token/internal"
    headers = {
        "Content-Type": "application/json; charset=utf-8",
    }
    payload_data = {
        "app_id": "cli_xxx",
        "app_secret": "xxx",
    }
    response = requests.post(url=url, data=json.dumps(payload_data), headers=headers).json()
    print(response)
    token = response['tenant_access_token']
    return token


In [ ]:
def upload_file(file_path):
    try:
        file_name = file_path.split(',')[0]
        url = "https://open.feishu.cn/open-apis/im/v1/files"
        form = {'file_type': 'stream',
                'file_name': file_name,
                'file': (file_name, open(file_path, 'rb'),
                         'text/plain')}  # 需要替换具体的path  具体的格式参考  https://www.w3school.com.cn/media/media_mimeref.asp
        multi_form = MultipartEncoder(form)
        headers = {'Authorization': f'Bearer {get_token()}',  ## 获取tenant_access_token, 需要替换为实际的token
                   }
        headers['Content-Type'] = multi_form.content_type
        r = requests.request("POST", url, headers=headers, data=multi_form)
        print(r.json())
        # print(response.headers['X-Tt-Logid'])  # for debug or oncall
        # print(response.content)  # Print Response
        if r.json().get("code") == 0 and r.json().get("msg") == "success":
            logger.info(f"上传文件到飞书成功，msg={r.json()},{file_path=}")
            media_id = r.json().get('data').get('file_key')
            return media_id
        else:
            logger.warning(f"上传文件到飞书异常,{r.json()=},{file_path=}")

        # Press the green button in the gutter to run the script.

    except Exception as e:
        # logger.warning("上传文件到企业微信失败")
        print("上传文件到飞书失败")
        print(e)
        # logger.warning(e)
        pass


In [ ]:
def get_qun_list():
    url = 'https://open.feishu.cn/open-apis/im/v1/chats'
    headers = {
        'Authorization': 'Bearer ' + get_token()
    }
    response = requests.get(url=url, headers=headers)
    print(response.json())
    return response.json()['data']['items']


def get_by_name(qun_name):
    items = get_qun_list()
    for i in items:
        if i.get('name') == str(qun_name):
            print(i.get('chat_id'))
            return i.get('chat_id')
